In [5]:
from typing import Optional
from pathlib import Path
from datetime import datetime, timedelta

import pandas as pd
import fire
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame

from src.path import DATA_DIR
from src.logger import get_console_logger
import src.config as cfg


In [8]:
cfg.APIkeys.alpaca_api_key

In [90]:

logger = get_console_logger(name='dataset_generation')

# Initialize Alpaca client
#client = StockHistoricalDataClient(cfg.APIkeys.alpaca_api_key, cfg.APIkeys.alpaca_secret_key)
client = StockHistoricalDataClient("PKSQNNG8GVINE9P4BOCI",  "G7PZXasCjXqnYJYB9QoSiF5iBYQSscn7zexMsfKx")
def download_ohlc_data_from_alpaca(
    symbol: Optional[str] = "NYSE",
    from_day: Optional[str] = "2023-01-01",
    to_day: Optional[str] = "2024-01-01",
) -> Path:
    """
    Downloads historical OHLC data from Alpaca API and saves data to disk
    """
    # create list of days as strings
    days = pd.date_range(start=from_day, end=to_day, freq="1D")
    
    # create empty dataframe
    data = pd.DataFrame()

    # create download dir folder if it doesn't exist
    download_dir = DATA_DIR / 'downloads'
    download_dir.mkdir(parents=True, exist_ok=True)
    
    for day in days:
        day_str = day.strftime("%Y-%m-%d")
        file_name = download_dir / f'{day_str}.parquet'
        if file_name.exists():
            logger.info(f'File {file_name} already exists, skipping')
            data_one_day = pd.read_parquet(file_name)
        else:
            logger.info(f'Downloading data for {day_str}')
            data_one_day = download_data_for_one_day(symbol, day_str)
            data_one_day.to_parquet(file_name, index=False)
        
        # combine today's file with the rest of the data
        data = pd.concat([data, data_one_day], ignore_index=True)

    # save data to disk   
    output_file = DATA_DIR / f"ohlc_data.parquet"
    data.to_parquet(output_file, index=False)

    return output_file

def download_data_for_one_day(symbol: str, day: str) -> pd.DataFrame:
    """
    Downloads one day of data and returns a pandas DataFrame
    """
    # Convert day string to datetime object
    start = datetime.strptime(day, "%Y-%m-%d")
    end = start + timedelta(days=1)

    # Request data from Alpaca
    request_params = StockBarsRequest(
        symbol_or_symbols=[symbol],
        timeframe=TimeFrame.Hour,
        start=start,
        end=end
    )
    bars = client.get_stock_bars(request_params)
    
    # Transform bars to pandas DataFrame
    data = bars.df.reset_index()
    return data[['timestamp', 'low', 'high', 'open', 'close', 'trade_count']].rename(columns={'timestamp':'time', 'trade_count':'volume'})


In [63]:
days = pd.date_range(start="2023-01-01", end="2024-06-01", freq="1D")

In [76]:
data_one_day

,index


In [ ]:
start = datetime.strptime(day, "%Y-%m-%d")
end = start + timedelta(days=1)

In [68]:
start = datetime.strptime("2024-06-01", "%Y-%m-%d")
end = start + timedelta(days=1)


In [70]:
start.isoformat()

'2024-06-01T00:00:00'

In [80]:
data = StockBarsRequest(symbol_or_symbols=["AAPL"],
timeframe=TimeFrame.Hour,
start=datetime(2022, 7, 1),
end=datetime(2022, 10, 1))

In [82]:
bars = client.get_stock_bars(data)

In [88]:
start = datetime.strptime("2023-06-01", "%Y-%m-%d")
end = start + timedelta(days=1)

# Request data from Alpaca
request_params = StockBarsRequest(
    symbol_or_symbols=["AAPL"],
    timeframe=TimeFrame.Hour,
    start=start,
    end=end
)
bars = client.get_stock_bars(request_params)

In [86]:
start

datetime.datetime(2024, 6, 1, 0, 0)

In [87]:
end

datetime.datetime(2024, 6, 2, 0, 0)

In [89]:
bars

{   'data': {   'AAPL': [   {   'close': 177.95,
                                'high': 177.95,
                                'low': 177.35,
                                'open': 177.8,
                                'symbol': 'AAPL',
                                'timestamp': datetime.datetime(2023, 6, 1, 8, 0, tzinfo=TzInfo(UTC)),
                                'trade_count': 738.0,
                                'volume': 27285.0,
                                'vwap': 177.625059},
                            {   'close': 177.73,
                                'high': 177.94,
                                'low': 177.56,
                                'open': 177.94,
                                'symbol': 'AAPL',
                                'timestamp': datetime.datetime(2023, 6, 1, 9, 0, tzinfo=TzInfo(UTC)),
                                'trade_count': 524.0,
                                'volume': 26675.0,
                                'vwap': 177.694505

In [ ]:

if __name__== '__main__':
    fire.Fire(download_ohlc_data_from_alpaca)